**Table of contents**<a id='toc0_'></a>    
- [표본추출](#toc1_)    
    - [단순 랜덤 추출법](#toc1_1_1_)    
    - [계통 추출법 (Systematic Sampling)](#toc1_1_2_)    
    - [집락 추출법 (Cluster Random Sampling)](#toc1_1_3_)    
    - [층화 추출법 (Stratified Random Sampling)](#toc1_1_4_)    
- [데이터 분할](#toc2_)    
    - [Hold-out](#toc2_1_1_)    
    - [K-Fold](#toc2_1_2_)    
    - [Stratified K-Fold](#toc2_1_3_)    
    - [Group K-Fold](#toc2_1_4_)    
- [교차 검증](#toc3_)    
    - [Cross Validation](#toc3_1_1_)    
    - [Grid Search Cross Validation](#toc3_1_2_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[표본추출](#toc0_)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams['font.size'] = 10
plt.rcParams['font.family'] = 'Malgeun Gothic'
plt.rcParams['figure.figsize'] = (5, 5)
plt.rcParams['axes.unicode_minus'] = False

%matplotlib inline

from sklearn.datasets import load_iris
data = load_iris()
iris_cols = list(data.feature_names) + ['target']
iris = pd.DataFrame(np.c_[data.data, data.target], columns=[col.replace(" (cm)", "") for col in iris_cols])

iris.head(3)

,sepal length,sepal width,petal length,petal width,target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0


### <a id='toc1_1_1_'></a>[단순 랜덤 추출법](#toc0_)

In [6]:
# random 라이브러리를 활용한 비복원 추출
import random
data_list = [1,2,3,4,5, 'a', 'b', 'c']
print(random.sample(data_list, 4))

# Numpy를 활용한 복원 추출
print(np.random.choice(data_list, 4, replace=True))

# Pandas를 활용한 비복원 추출 - 개수 지정 방식
iris.sample(n=3, replace=False)

# Pandas를 활용한 비복원 추출 - 비율 지정 방식
iris.sample(frac=0.3)

# Pandas를 활용한 비복원 추출 - 특정 열에 가중치 적용 (sepal length가 클수록 당첨확률 커짐)
testdf = iris.sample(frac=0.3, weights='sepal length')

testdf[:3]

[1, 'a', 'c', 'b']
['3' '1' '1' '1']


,sepal length,sepal width,petal length,petal width,target
137,6.4,3.1,5.5,1.8,2.0
27,5.2,3.5,1.5,0.2,0.0
86,6.7,3.1,4.7,1.5,1.0


### <a id='toc1_1_2_'></a>[계통 추출법 (Systematic Sampling)](#toc0_)
- 특정 feature 혹은 index를 기준으로 샘플을 나열한다.
- N개의 샘플 중 n개를 추출할 예정이라면, N/n 개로 구간을 분할한다.
- 각 구간 별로 1개씩 임의로 선정한다.

In [4]:
data, n = iris, 8
rows = data.shape[0]
k = int(rows / 8)
beg = 0 
end = k

sys_df = pd.DataFrame()

while end <= rows :
    print(beg, end)
    sys_df = pd.concat([sys_df, data.loc[beg:end,].sample(1)])
    beg = end
    end += k
    

sys_df


0 18
18 36
36 54
54 72
72 90
90 108
108 126
126 144


,sepal length,sepal width,petal length,petal width,target
12,4.8,3.0,1.4,0.1,0.0
34,4.9,3.1,1.5,0.2,0.0
42,4.4,3.2,1.3,0.2,0.0
62,6.0,2.2,4.0,1.0,1.0
72,6.3,2.5,4.9,1.5,1.0
94,5.6,2.7,4.2,1.3,1.0
121,5.6,2.8,4.9,2.0,2.0
136,6.3,3.4,5.6,2.4,2.0


### <a id='toc1_1_3_'></a>[집락 추출법 (Cluster Random Sampling)](#toc0_)
- 군집을 설정하고 각 군집(cluster) 별로 랜덤 추출법 (이 부분은 층화추출과 동일)
- 1000명의 유권자를 먼저 선정한 다음 그 안에서 랜덤 추출. 학교를 3곳 먼저 선정한 다음 그 안에서 랜덤 추출
- 집단 간 동질성, 집단 내 이질성이 특징

### <a id='toc1_1_4_'></a>[층화 추출법 (Stratified Random Sampling)](#toc0_)
- 층을 설정하고 각 층(stratum) 별로 랜덤 추출 (이 부분은 집락추출과 동일)
- 남성군집과 여성군집을 먼저 설정한 다음 그 안에서 램덤 추출. 1학년, 2학년, 3학년 군집을 먼저 설정한 다음 그 안에서 랜덤 추출
- 집단 간 이질성, 집단 내 동질성이 특징

In [69]:
#아래에서는 비례 층화추출법과 불비례 층화추출법을 구현한다.
import math

def iris_StratRandSampling(method, data, samples, props):
    count = data.target.size
    level = data.target.nunique()
    levels = data.target.value_counts()
    
    lev_count = []
    if method == '비례' :
        lev_count = (levels / count)*samples
    elif method == '불비례':
        temp = []
        for i in range(len(props)) :
            temp.append(props[i]*samples)
        lev_count = [math.ceil(temp[i]) for i in range(len(temp))]  #props로 계산 시 올림으로 개수 보정한다.

    srs_df = pd.DataFrame()
    levels = list(map(int, levels.index.to_list()))             #index로 사용하기 위해 정수로 변환해둔다.
    
    for idx in levels:
        df = data[data.target==idx].sample(int(lev_count[idx]))
        srs_df = pd.concat([srs_df, df])

    return srs_df

In [71]:
iris_StratRandSampling('비례', iris, 9, [0.2, 0.3, 0.5])

,sepal length,sepal width,petal length,petal width,target
9,4.9,3.1,1.5,0.1,0.0
23,5.1,3.3,1.7,0.5,0.0
28,5.2,3.4,1.4,0.2,0.0
67,5.8,2.7,4.1,1.0,1.0
70,5.9,3.2,4.8,1.8,1.0
66,5.6,3.0,4.5,1.5,1.0
136,6.3,3.4,5.6,2.4,2.0
115,6.4,3.2,5.3,2.3,2.0
105,7.6,3.0,6.6,2.1,2.0


# <a id='toc2_'></a>[데이터 분할](#toc0_)

### <a id='toc2_1_1_'></a>[Hold-out](#toc0_)
- 하나의 데이터셋을 가지고 학습을 하고 검증을 하면 성능이 좋게 나올 수 밖에 없다.
- 데이터셋을 2개의 덩어리로 나누어 하나는 학습용, 하나는 검증용으로 사용하는 것을 Hold-out이라고 한다.
- 보통은 7:3의 비율로 학습용과 검증용을 나누어 사용한다.

In [3]:
from sklearn.model_selection import train_test_split

X = iris.drop(['target'], axis=1)
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

print(len(X_train), len(X_test))

105 45


### <a id='toc2_1_2_'></a>[K-Fold](#toc0_)
- Hold-out 데이터 분할할 경우 분할이 어떻게 되느냐에 따라 과적합 혹은 과소적합이 될 수 있다.
- 이로 인해 데이터를 n 개의 덩어리(Fold)로 나누고, 그 중 1개는 검증용, 나머지를 학습용으로 사용하는 처리를 n번 반복하게 된다.
- 각 회차에서 나온 MSE의 평균을 해당 모델의 MSE로 채택한다.

### <a id='toc2_1_3_'></a>[Stratified K-Fold](#toc0_)
- K-Fold 데이터 분할할 경우, 각 Fold에는 target 값이 균등하게 배분되지 않을 가능성이 높고, 이로 인해 과대적합, 과소적합 우려가 있다.
- 따라서 K-Fold를 만들 때, 각 Fold에 target 값의 구성비에 따라 균등하게 배분되도록 Fold를 구성하는 것이 적절하다.

### <a id='toc2_1_4_'></a>[Group K-Fold](#toc0_)
- Stratified에서는 target의 범주가 3이라고 하면, 각 Fold에 3개 범주의 데이터가 동일 비율로 포함되도록 하는 것을 목적으로 한다.
- 그런데 반대로 한 유형의 데이터는 하나의 Fold에만 속하도록 해야 할 때가 있다. 
- 이를 테면 여러 환자의 데이터가 담겨 있는 경우, 학습용으로는 A, B 환자의 데이터만 사용하고 검증용으로는 C, D 환자의 데이터를 사용하고 싶은 경우다.
- 이런 경우 환자의 데이터가 학습용에 있었다면, 검증용으로 사용되면 안되며, 이런 경우 사용하는 것이 Group K-Fold이다.

In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold, StratifiedGroupKFold 
from collections import Counter

kfold = KFold(n_splits=4, shuffle=False)    #shuffle은 분할 전 섞기
print('K-Fold')
for i, (train_index, test_index) in enumerate(kfold.split(X)) :
    y_train = y[train_index]
    y_test = y[test_index]
    print(train_index.size, Counter(y_train), test_index.size, Counter(y_test))


skfold = StratifiedKFold(n_splits=4, shuffle=False)
print('Stratified K-Fold')
for i, (train_index, test_index) in enumerate(skfold.split(X, y)) :
    y_train = y[train_index]
    y_test = y[test_index]
    print(train_index.size, Counter(y_train), test_index.size, Counter(y_test))


gkfold = GroupKFold(n_splits=4)
print('Group K-Fold')
# Group K-Fold 테스트를 위한 group열 생성
X['group'] = y.apply(lambda x : f'g{int(np.random.randint(0,4,1))}')
for i, (train_index, test_index) in enumerate(gkfold.split(X, y, X.group)) :
    g_train = X.group[train_index]
    g_test = X.group[test_index]
    print(len(train_index), Counter(g_train), len(test_index), Counter(g_test))


# <a id='toc3_'></a>[교차 검증](#toc0_)

In [24]:
X = iris.drop(['target'], axis=1)
y = iris.target

### <a id='toc3_1_1_'></a>[Cross Validation](#toc0_)
- 데이터 분할에 따른 학습을 동시에 처리해준다.

In [25]:
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.linear_model import LogisticRegression

skfold = StratifiedKFold(n_splits=4, shuffle=False)
lr1_model = LogisticRegression(max_iter=300, C=0.1)
result = cross_validate(lr_model1, X, y, cv=skfold, return_train_score=True)

print(pd.DataFrame(result))

   fit_time  score_time  test_score  train_score
0  0.027629    0.006762    0.894737     0.955357
1  0.025223    0.003410    0.947368     0.964286
2  0.026812    0.004243    0.945946     0.955752
3  0.032816    0.005445    1.000000     0.938053


### <a id='toc3_1_2_'></a>[Grid Search Cross Validation](#toc0_)
- 데이터분할 및 학습을 해주는데, 각 파라미터 조합마다 한번씩 해준다.

In [27]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.linear_model import LogisticRegression

skFold = StratifiedKFold(n_splits=4, shuffle=False)
lr2_model = LogisticRegression(max_iter=300)
param_grid = {'C':[0.01, 0.1, 1], 'solver':['lbfgs', 'liblinear']}

grid = GridSearchCV(lr2_model, param_grid, cv=skfold)
grid.fit(X, y)

print('최고 점수: {:.2f}'.format(grid.best_score_))
print('최적 Param: {}'.format(grid.best_params_))

최고 점수: 0.97
최적 Param: {'C': 1, 'solver': 'lbfgs'}
